In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
)
def predict_category(query):
    prompt = f"""
    다음은 사용자의 질문입니다. 질문을 아래의 카테고리 중 하나로 분류해주세요:
    - 경마방법: 경마 규칙, 배팅 방법, 경마 용어 등과 관련된 질문.
    - 경기스케쥴: **날짜, 경마 일정, 시간표, 장소(예: 서울 경마공원 일정, 부산 경마공원 일정 등),** 2024년 경기와 관련된 질문. 예) "12월 21일 서울 경주 일정은?", "2024년 부산 경마 일정은?"
    - 우승마기록: 우승 기록, 인기 있는 말, 말의 성적 등과 관련된 질문.
    - 2024시행계획: 2024년 관련 계획, 정책, 새로운 규정 등과 관련된 질문.
    - 기타: 위의 카테고리에 해당하지 않는 질문.
    
    위 카테고리 중 가장 적합한 하나를 선택해서 답변해주세요. 위 카테고리에 포함되지 않으면 "기타"입니다.
    서울 경주 일정을 물어보면 경기스케줄로 분류하세요
"""
    response = model.invoke(prompt)
    category = response.content.strip()

    # 유효한 카테고리 확인
    valid_categories = ['경마방법', '경기스케쥴', '우승마기록', '2024시행계획']
    return category if category in valid_categories else "기타"

category_to_abbreviation = {
    '경마방법': 'guide',
    '경기스케쥴': 'schedule',
    '우승마기록': 'winners',
    '2024시행계획': 'plan2024',
    '기타': 'all', 
}

vectorstore_folder = 'C:\\Workspace\\DA36_mini4_ma\\min\\vectorstores'
def ask_question_with_rag(query):
    predicted_category = predict_category(query)
    abbreviation = category_to_abbreviation.get(predicted_category, 'all')
    vectorstore_path = os.path.join(vectorstore_folder, f"vs_{abbreviation}")

    vector_store = FAISS.load_local(
        vectorstore_path, 
        OpenAIEmbeddings(model="text-embedding-ada-002"), 
        allow_dangerous_deserialization=True
    )
    retriever = vector_store.as_retriever()

    rqa = RetrievalQA.from_chain_type(
        llm=model,
        retriever=retriever,
        chain_type="stuff"
    )
    response = rqa({"query": query})
    return response['result']

### 카테고리 분류 질문

In [8]:
# 카테고리 질문
query="2024년 12월 21일 서울 1경주 경주력을 분석해서 배팅할만한 말 세 마리만 말해줘"
response = predict_category(query)
print(response)

경기스케쥴


### 테스트 QnA

In [11]:
query="2024년 12월 21일 서울 1경주의 말들을 알려줘"
print(ask_question_with_rag(query))

2024년 12월 21일 서울 1경주에 출전하는 말들은 다음과 같습니다:

1. 관악산원더풀
2. 스피드레전드
3. 남산소녀
4. 굿프렌드
5. 혁명의질주
6. 신한에이스
7. 레이디마고
8. 파이널리더
9. 미스페이머스
10. 키아오라
11. 이슬처럼


In [12]:
query="2024년 서울에서 가장 많이 우승한 말을 알려줘"
print(ask_question_with_rag(query))

알 수 없습니다.
